# ETL BBDD ECHA
El objetivo de esta ETL es **extraer** el listado de Disruptores Endocrinos de la ECHA (European Chemicals Agency) https://echa.europa.eu/es/ed-assessment **transformar** la BBDD para luego **cargarla** junto con la Edlist y unir ambos dataframes.

El motivo de evaluar esta BBDD y no solo quedarnos con la EDlist, en la página de la Edlist se informa:
https://edlists.org/ed-relevant-legislations

" Más de 40 legislaciones diferentes intervienen en la regulación de sustancias químicas en la UE. Aquí se ofrece una breve descripción de las jurisdicciones más relevantes para la regulación de sustancias químicas disruptoras endocrinas, así como de los procedimientos correspondientes.
- Organismo responsable : ECHA (Agencia Europea de Sustancias y Mezclas Químicas)
- Jurisdicción : REACH regula los productos químicos industriales producidos o importados en la UE.

Procedimiento : Si un Estado miembro o la ECHA tienen una preocupación debido a las propiedades de alteración endocrina de un compuesto, pueden proponerlo como SVHC (Sustancia Extremadamente Preocupante) de acuerdo con el artículo 57(f) del reglamento REACH. Tras una ronda de comentarios públicos, la propuesta se remite al Comité de los Estados Miembros (MSC) para la búsqueda de un acuerdo. Si el MSC alcanza una decisión unánime, el compuesto se incluye en la Lista de Sustancias Candidatas de Muy Alta Preocupación para su Autorización debido a sus propiedades de alteración endocrina. Si los países del MSC no están de acuerdo, la propuesta se remite a la Comisión Europea para una decisión final.

(...)

La ECHA evalúa periódicamente las sustancias de la Lista de Candidatos para decidir cuáles deben incluirse en la Lista de Autorización con carácter prioritario. "

Por ese motivo, consideramos que para cubrir un mayor campo, hay que tener en cuenta ambos listados y en el peor de los caso eliminar duplicados.

La ECHA explica en su web (enlace anterior) en el apartado "Explanatory note":

"La inclusión en la lista significa que se está desarrollando o se ha completado desde febrero de 2013 una evaluación informal de riesgos para las propiedades disruptoras endocrinas.

Para cada sustancia, la tabla muestra el Estado miembro evaluador (presentador), el resultado de la evaluación y la fecha de la última actualización de la entrada de la lista."

Por ese motivo, tendrémos en cuenta las variables "Status" y "Outcome" en donde en combinación de ambas variables se manifiesta si es disruptor confirmado o en evaluación

## 1. Importamos librerias

In [1]:
import pandas as pd
import numpy as np

## 2. Lectura de datos e información de la lista ECHA

In [2]:
# leemos los datos de la lista de ECHA
echa= pd.read_excel("../../data/raw/endocrine-disruptor-assessment-export.xlsx", skiprows=3)


b:\IMF\TFM\tfm-disruptors\tfm_env\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
print("""
Nº de filas del dataset ECHA: {}
Nº de columnas del dataset ECHA: {}
Las variables de ECHA son: {}
""".format(echa.shape[0], echa.shape[1], echa.columns.tolist()))


Nº de filas del dataset ECHA: 177
Nº de columnas del dataset ECHA: 9
Las variables de ECHA son: ['Substance name', 'DISLIST_ED_description', 'EC / List no', 'CAS no', 'Authority', 'Status', 'Outcome', 'Latest update', 'First published']



In [4]:
echa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Substance name          177 non-null    object
 1   DISLIST_ED_description  9 non-null      object
 2   EC / List no            177 non-null    object
 3   CAS no                  177 non-null    object
 4   Authority               176 non-null    object
 5   Status                  177 non-null    object
 6   Outcome                 177 non-null    object
 7   Latest update           177 non-null    object
 8   First published         177 non-null    object
dtypes: object(9)
memory usage: 12.6+ KB


Solo hay 1 nulo en **Authority** y casi todos los de la variable **DISLIST_ED_description** son nulos.

El resto de variables parecen estar completas.

**Procedemos a su eliminación "DISLIST_ED_description"**
Porque no contiene apenas información y además aunque en el archivo aparezca en la web no aparece.

In [5]:
echa.drop(columns='DISLIST_ED_description', inplace=True)

In [6]:
# Renombramos variable para que coincida con la Edllist
echa.rename(columns={'CAS no': 'CAS Number', 'EC / List no':'EC Number'}, inplace=True)

## 3. Duplicados

In [7]:
# Para hacer la comparativa de duplicados vamos a elegir la variable "Name and abbreviation" pues es única para cada ingrediente y no tiene nulos.
duplicados = echa[echa.duplicated(subset=['Substance name', 'CAS Number', 'EC Number' ], keep=False)]
duplicados

,Substance name,EC Number,CAS Number,Authority,Status,Outcome,Latest update,First published
16,"4,4'-isopropylidenediphenol",201-245-8,80-05-7,Germany,Concluded,ED ENV,03-feb-2021,12-oct-2018
17,"4,4'-isopropylidenediphenol",201-245-8,80-05-7,France,Concluded,ED HH,03-feb-2021,12-oct-2018
18,"4,4'-methylenedi-2,6-xylenol",226-378-9,5384-21-4,France,Under development,Under development (other),03-feb-2021,12-oct-2018
52,Dicyclohexyl phthalate,201-545-9,84-61-7,Sweden,Concluded,ED HH,03-feb-2021,04-oct-2018
57,"Phenol, dodecyl-, branched",310-154-3,121158-58-5,Germany,Concluded,ED ENV#ED HH,03-feb-2021,12-oct-2018
58,"resorcinol; 1,3-benzenediol",203-585-2,108-46-3,Finland,Concluded,inconclusive,03-feb-2021,12-oct-2018
59,"resorcinol; 1,3-benzenediol",203-585-2,108-46-3,France,Under development under SEV,Under development (SEV),03-feb-2021,25-oct-2018
60,"resorcinol; 1,3-benzenediol",203-585-2,108-46-3,France,Under development,Under development (other),03-feb-2021,06-nov-2019
81,"Phenol, dodecyl-, branched",310-154-3,121158-58-5,Germany,Concluded,ED ENV#ED HH,27-sep-2021,19-oct-2020
100,"(3E)-1,7,7-trimethyl-3-(4-methylbenzylidene)bi...",701-394-3,1782069-81-1,Denmark,Concluded,ED HH,04-ago-2022,20-oct-2020


In [8]:
# Eliminamos los duplicados por las variables más relantes y ordenando por las otras que pierdan información
echa = echa.drop_duplicates(subset=['Substance name', 'CAS Number', 'EC Number' ], keep='first').sort_values(by = 'Authority')

In [9]:
echa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168 entries, 46 to 170
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Substance name   168 non-null    object
 1   EC Number        168 non-null    object
 2   CAS Number       168 non-null    object
 3   Authority        167 non-null    object
 4   Status           168 non-null    object
 5   Outcome          168 non-null    object
 6   Latest update    168 non-null    object
 7   First published  168 non-null    object
dtypes: object(8)
memory usage: 11.8+ KB


## 4. Comprobación de "EC Number" y "CAS Number" - Mismo procedimiento que para EDlist

Las dos variables que coinciden en ambos dataframes son "EC Number" y "CAS Number", por tanto vamos a comprobar el estado de esas variables en el dataframe ECHA:

Comprobamos si hay más de un "EC Number" y "CAS Number" en el mismo registro, como pasaba con la Edlist

In [10]:
# Filtrar filas con más de un CAS no. de ECHA
echa[echa['CAS Number'].str.contains(r'[/;,]', regex= True,na=False)]

# En este dataframe no hay problemas

,Substance name,EC Number,CAS Number,Authority,Status,Outcome,Latest update,First published


In [11]:
# Filtrar filas con más de un CAS no. de ECHA
echa[echa['EC Number'].str.contains((r'[,;/]'), na=False)]

# En este dataframe no hay problemas

,Substance name,EC Number,CAS Number,Authority,Status,Outcome,Latest update,First published


In [12]:
echa[echa['CAS Number'] == '-']

,Substance name,EC Number,CAS Number,Authority,Status,Outcome,Latest update,First published
165,Reaction products of paraformaldehyde and 2-hy...,-,-,Austria,Concluded,inconclusive,13-ene-2025,02-jun-2021
166,reaction products of paraformaldehyde with 2-h...,-,-,Austria,Concluded,inconclusive,13-ene-2025,02-jun-2021
19,"4-heptylphenol, branched and linear",-,-,Austria,Concluded,ED ENV,03-feb-2021,04-oct-2018
98,"(±)-1,7,7-trimethyl-3-[(4-methylphenyl)methyle...",-,-,Denmark,Concluded,ED HH,04-ago-2022,20-oct-2020
142,Oligomerisation and alkylation reaction produc...,700-960-7,-,Denmark,Concluded,ED ENV,04-jun-2024,25-oct-2018
141,Mono- and di-phthalate esters with linear and/...,-,-,France,Under development,Under development (other),18-abr-2024,18-abr-2024
150,Reaction mass of p-t-butylphenyldiphenyl phosp...,939-505-4,-,France,Under development under SEV,Under development (SEV),16-ago-2024,15-may-2024
176,Reaction mass of 3-[(diphenoxyphosphoryl)oxy]p...,701-337-2,-,France,Concluded,inconclusive,26-jun-2025,12-oct-2018
7,"1-(5,6,7,8-tetrahydro-3,5,5,6,8,8-hexamethyl-2...",-,-,Germany,Under development under SEV,Under development (SEV),03-feb-2021,06-nov-2019
118,"Reaction mass of 2,6-bis(1-phenylethyl)phenol ...",701-171-0,-,Germany,Concluded,ED ENV,20-mar-2023,23-mar-2022


Vemos que ambas variables "EC Number" y "CAS Number" contienen valores: "-" por lo que los sustituiremos por NAN

In [13]:
echa['CAS Number'] = echa['CAS Number'].replace(['-', '', ' '], np.nan)
echa['EC Number'] = echa['EC Number'].replace(['-', '', ' '], np.nan)

In [14]:
echa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168 entries, 46 to 170
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Substance name   168 non-null    object
 1   EC Number        134 non-null    object
 2   CAS Number       157 non-null    object
 3   Authority        167 non-null    object
 4   Status           168 non-null    object
 5   Outcome          168 non-null    object
 6   Latest update    168 non-null    object
 7   First published  168 non-null    object
dtypes: object(8)
memory usage: 11.8+ KB


Vemos que hemos pasado de no tener nulos en las variables "EC no." y "CAS no." ahora tenemos algunos.
Tambien vemos que en la consulta anterior hay dos registros que tienen en nulo en las 2 varibles que por las que vamos a hacer la unión: "CAS no." y "EC no."
Hay que eliminar para no crear conflictos en la unión.

In [15]:
# Filtramos los registros que umplen la condición que "CAS no." y "EC no." sean NO nulos.

echa_clean = echa[~(echa['CAS Number'].isna() & echa['EC Number'].isna())]

In [16]:
echa_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162 entries, 46 to 170
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Substance name   162 non-null    object
 1   EC Number        134 non-null    object
 2   CAS Number       157 non-null    object
 3   Authority        161 non-null    object
 4   Status           162 non-null    object
 5   Outcome          162 non-null    object
 6   Latest update    162 non-null    object
 7   First published  162 non-null    object
dtypes: object(8)
memory usage: 11.4+ KB


## 5. Creación variable fuente_original

Vamos a crear esta variable con la información según el disruptor está en evaluación o está confirmado

In [17]:
echa_clean['Status'].value_counts()

Status
Concluded                      104
Under development under SEV     29
Under development               29
Name: count, dtype: int64

In [18]:
echa_clean['Outcome'].value_counts()

Outcome
ED ENV                       40
Under development (SEV)      29
Under development (BPR)      22
postponed                    13
ED ENV#ED HH                 13
inconclusive                 12
ED HH                        11
not ED                       10
Under development (other)     7
ED ENV#inconclusive           2
not ED#inconclusive           2
ED ENV 1                      1
Name: count, dtype: int64

La información de la variable Status se amplia en la variable Outcome, que es en donde se dice cómo se ha concluido la investigación.

Por ello, vamos consultar qué valores toma **Outcome** cuando la variable **Status** toma valor concluide y cuando no

In [19]:
Status_concluded = echa_clean[echa_clean['Status']== 'Concluded']
Status_concluded['Outcome'].value_counts()

Outcome
ED ENV                 40
postponed              13
ED ENV#ED HH           13
inconclusive           12
ED HH                  11
not ED                 10
ED ENV#inconclusive     2
not ED#inconclusive     2
ED ENV 1                1
Name: count, dtype: int64

In [20]:
Status_no_concluded = echa_clean[echa_clean['Status']!= 'Concluded']
Status_no_concluded['Outcome'].value_counts()

Outcome
Under development (SEV)      29
Under development (BPR)      22
Under development (other)     7
Name: count, dtype: int64

Con esta información vamos a mapear outcome para darle valores: 
* ED_Env
* ED_Humans
* ED_Humans&Env
* ED_pendiente

Tendremos en cuenta como pendiente inconclusive e postponed, porque si bien es cierto que están dentro del grupo de **concluded**, no se ha descartado ni confirmado como disruptor.

Pero antes vamos a elimar los que no son ED porque al estar confirmados que no lo son no nos interesa tenerlos

In [21]:
echa_clean= echa_clean[~echa_clean['Outcome'].isin(['not ED', 'not ED#inconclusive'])]
echa_clean['Outcome'].value_counts()

Outcome
ED ENV                       40
Under development (SEV)      29
Under development (BPR)      22
postponed                    13
ED ENV#ED HH                 13
inconclusive                 12
ED HH                        11
Under development (other)     7
ED ENV#inconclusive           2
ED ENV 1                      1
Name: count, dtype: int64

In [22]:
# Creamos un diccionario de mapeo
mapeo = {
    "ED ENV": "ECHA - ED_Env",
    "ED ENV#ED HH": "ECHA - ED_Humans&Env",
    "ED HH": "ECHA - ED_Humans",
    "ED ENV 1": "ECHA - ED_Env",
    "Under development (SEV)" : "ECHA - ED_pendiente",
    "Under development (BPR)" : "ECHA - ED_pendiente",
    "inconclusive" : "ECHA - ED_pendiente",
    "postponed" : "ECHA - ED_pendiente",
    "Under development (other)" : "ECHA - ED_pendiente",
    "ED ENV#inconclusive" : "ECHA - ED_pendiente"
}

# Creamos la nueva columna
echa_clean['Fuente_original'] = echa_clean['Outcome'].map(mapeo)


In [23]:
echa_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150 entries, 46 to 170
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Substance name   150 non-null    object
 1   EC Number        122 non-null    object
 2   CAS Number       145 non-null    object
 3   Authority        149 non-null    object
 4   Status           150 non-null    object
 5   Outcome          150 non-null    object
 6   Latest update    150 non-null    object
 7   First published  150 non-null    object
 8   Fuente_original  150 non-null    object
dtypes: object(9)
memory usage: 11.7+ KB


In [24]:
echa_clean['Fuente_original'].value_counts()

Fuente_original
ECHA - ED_pendiente     85
ECHA - ED_Env           41
ECHA - ED_Humans&Env    13
ECHA - ED_Humans        11
Name: count, dtype: int64

## 6. Exportamos base de datos limpia
Posteriormente vamos a unir esta tabla con la tabla EDlist y CosIng.

In [25]:
# Reiniciamos ínfice
echa_clean.reset_index(drop=True, inplace=True)

In [26]:
# CSV
# echa_clean.to_csv("../../data/processed/notebooks/echa_clean.csv", index=False)

# Parquet
echa_clean.to_parquet("../../data/processed/notebooks/echa_clean.parquet", index=False)